In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# import data 
data = pd.read_csv('Data/ImprovedAirbnbCleaned.csv')
data.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
Bronx                               int64
Brooklyn                            int64
Manhattan                           int64
Queens                              int64
Staten Island                       int64
days_since_last_review            float64
Entire home/apt                     int64
Private room                      